In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt

%matplotlib inline

# To avoid warning messages
import warnings
warnings.filterwarnings('ignore')

### Local functions

In [ ]:
def get_real_data_fName(year, date_yyyymmdd, path_real="/discover/nobackup/projects/gmao/advda/sakella/future_sst_fraci/GMAO_OPS_bin_data/data/", pref="sst_ice_", suff=".nc"):
    fName = path_real + str(year) + "/" + pref + "{}".format(date_yyyymmdd) + suff
    return fName

def get_clim_data_fName(date_mmdd, path_clim="/discover/nobackup/projects/gmao/advda/sakella/future_sst_fraci/data/ncFiles/", pref="daily_clim_mean_sst_fraci_", suff=".nc"):
    fName = path_clim + "/" + pref + "0001{}".format(date_mmdd) + suff
    return fName

def get_sst(data_arr, lat0, lon0):
    data_ij = data_arr.sel(lat=lat0, lon=lon0, method='nearest').values
    return data_ij

## User inputs

In [ ]:
# Year and name of storm (in that year)
year = 2023

tc_name = 'franklin'
#tc_name = 'idalia'
#tc_name = 'lee'

fcst_nDays = 10 # 10-day forecasts

## Read pre-processed storm info.

In [ ]:
ds_tc= xr.open_dataset(tc_name+ str(year)+'.nc')

start_date = str(ds_tc['time'][0].dt.strftime("%Y-%m-%d").values)
end_date = str(ds_tc['time'][-1].dt.strftime("%Y-%m-%d").values)

print("\nHurricane:\t{} originated on:\t{},\t dissipated on:\t{}.\n".format(tc_name.upper(), start_date, end_date))

## Get the OSTIA SST along the TC track.
- The TC _moves_! 
- Therefore at any time, it has a specific location. 
- At that coordinate (time, position), we know the true SST (because we are in hindcast mode).

In [ ]:
real_sst = np.zeros( (ds_tc['time'].shape[0]), dtype=np.float32)

for id in range(0, ds_tc['time'].shape[0]):
    fName = get_real_data_fName(year, str(ds_tc['time'][id].dt.strftime("%Y%m%d").values))
    real_sst[id] = get_sst( xr.open_dataset(fName).SST, ds_tc['lat'].values[id], ds_tc['lon'].values[id])

ds_tc = ds_tc.assign(sst=(['time'],real_sst)) # add SST to dataset- make life easy! 

## Plot OSTIA SST along TC track

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))
ax2 = ax1.twinx()

ds_tc.sst.plot.line(ax=ax1,  c='b', ls='-', label='OSTIA SST')
ds_tc.mslp.plot.line(ax=ax2, c='r', ls='-', label='MSLP')

ax1.set_ylabel('SST ($^\circ$K)',  c='b', fontsize=14, weight='bold')
ax1.tick_params(axis='y', labelcolor='b')

ax2.set_ylabel('MSLP (hPa)', c='r', fontsize=14,weight='bold')
ax2.tick_params(axis='y', labelcolor='r')

ax1.set_title('{} {}'.format(tc_name.upper(),year), fontsize=15)

## Get predicted SST as the TC advances in time.
### From any given date, we _predict_ into future using `2` different methods:
1. `Persist` (from a starting date) SST into future
2. `Persist the starting date SST anomaly` (from a daily climatology) into future

As the TC advances in time, data for _past_ days is filled with _real_ data.

In [ ]:
exp_dates  = pd.date_range(str(start_date), end_date, freq='D') # Forecast from: TC start date to the end date
nfcst = exp_dates.shape[0]

predicted_sst_method1 = np.full( (nfcst, ds_tc['time'].shape[0]), np.nan, dtype=np.float32)
predicted_sst_method2 = np.full( (nfcst, ds_tc['time'].shape[0]), np.nan, dtype=np.float32)

for ifcst in range(1, nfcst+1): # each forecast
    # Each forecast is [fcst_nDays] = 10-days long
    fcst_dates = pd.date_range(start=exp_dates[0]+pd.DateOffset(days=ifcst-1), periods=fcst_nDays)
    fcst_start_date = fcst_dates[0].strftime("%Y%m%d")
    print("Forecast [{}] start date: {}".format(ifcst,fcst_start_date))
    
    index_start = np.where(ds_tc.time > fcst_dates[0])[0][0]
    past_indx   = np.arange(0, index_start) # --> real SST
    future_indx = np.arange(index_start, ds_tc['time'].shape[0]+1) # --> predicted SST
    
    #print("PAST", past_indx) # --> We know it.
    for id in range(0, past_indx.shape[0]):
        predicted_sst_method1[ifcst-1, id] = real_sst[id]
        predicted_sst_method2[ifcst-1, id] = real_sst[id]

    #print("FUTURE", future_indx[0], future_indx[-1]) # --> Unknown, predict SST.
    for id in range(future_indx[0], future_indx[-1]):
        if id == future_indx[0]:
            fName0_real = get_real_data_fName(year, str(ds_tc.time[id].dt.strftime("%Y%m%d").values))
            SST0 = xr.open_dataset(fName0_real).SST # Initial SST
            
            # Anomaly in SST on start date
            fName0_clim = get_clim_data_fName(str(ds_tc.time[id].dt.strftime("%m%d").values))
            clim_SST0 = xr.open_dataset(fName0_clim).SST
            
            dSST0=xr.full_like(clim_SST0, np.nan) # xarray is unhappy with the date in climatology file!
            dSST0.values = (SST0.values- clim_SST0.values) # Initial anomaly    
  
        fName_clim = get_clim_data_fName(str(ds_tc.time[id].dt.strftime("%m%d").values))
        clim_sst = xr.open_dataset(fName_clim).SST # daily climatology of the day
            
        predicted_sst_method1[ifcst-1, id]= get_sst( SST0, ds_tc['lat'].values[id], ds_tc['lon'].values[id])
        predicted_sst_method2[ifcst-1, id]= (clim_sst.squeeze()+dSST0.squeeze()).sel(lat=ds_tc['lat'].values[id],lon= ds_tc['lon'].values[id], method='nearest')

## Plot real and predicted data

In [ ]:
plt.figure( figsize=(16,8))

ds_tc.sst.plot.line(ls='-', lw=4, _labels=False, add_legend=True, label='OSTIA SST')
for ifcst in range(0, 1):#nfcst):
    plt.plot_date(ds_tc.time, predicted_sst_method1[ifcst,:], ls='-', marker=None, label='persist SST FCST {}'.format(str(ifcst+1).zfill(2)))
    plt.plot_date(ds_tc.time, predicted_sst_method2[ifcst,:], ls='-', marker=None, label='persist SSTanom FCST {}'.format(str(ifcst+1).zfill(2)))
plt.legend(ncol=5)
plt.ylabel('SST [$^\circ$K]', fontsize=12)

## Plot real and predicted data with MSLP

In [ ]:
fig, ax1 = plt.subplots(figsize=(16,8))

ax2 = ax1.twinx()

ds_tc.sst.plot.line(ax=ax1,  ls='-', lw=4, _labels=False, add_legend=True, label='OSTIA SST')
ds_tc.mslp.plot.line(ax=ax2, c='r', ls='-', label='MSLP')

# forecasts
for ifcst in range(0, 2):#nfcst):
    ax1.plot_date(ds_tc.time, predicted_sst_method1[ifcst,:], ls='--', marker=None, label='persist SST FCST {}'.format(str(ifcst+1).zfill(2)))
    ax1.plot_date(ds_tc.time, predicted_sst_method2[ifcst,:], ls='-', marker=None, label='persist SSTanom FCST {}'.format(str(ifcst+1).zfill(2)))

ax1.set_ylabel('SST ($^\circ$K)',  c='b', fontsize=14, weight='bold')
ax1.tick_params(axis='y', labelcolor='b')

ax2.set_ylabel('MSLP (hPa)', c='r', fontsize=14,weight='bold')
ax2.tick_params(axis='y', labelcolor='r')

ax1.set_title('{} {}'.format(tc_name.upper(),year), fontsize=15)

ax1.legend(ncol=1, fontsize=15)

## Compare different prediction methods:
- OSTIA SST.
- Plot forecast error mean and standard deviation.

In [ ]:
err1 = np.zeros_like(predicted_sst_method1)
err2 = np.zeros_like(predicted_sst_method2)

for ifcst in range(0, err1.shape[0]):
    err1[ifcst, :] = predicted_sst_method1[ifcst,:] - real_sst
    err2[ifcst, :] = predicted_sst_method2[ifcst,:] - real_sst
 
plt.figure( figsize=(16,10))

ax1=plt.subplot(311)
ax2 = ax1.twinx()
ax1.plot_date(ds_tc.time,  ds_tc.sst, ls='-', marker=None, label='OSTIA SST')
ax2.plot_date(ds_tc.time,  ds_tc.mslp, ls='--', marker=None, label='MSLP')
plt.title('{} {}'.format(tc_name.upper(),year), fontsize=15)
ax1.legend(loc=3); ax2.legend(loc=1)
ax1.set_ylabel('SST ($^\circ$K)')
ax2.set_ylabel('MSLP (hPa))')
#
plt.subplot(312)
plt.plot_date( ds_tc.time, np.mean(err1, axis=0, dtype=np.float64), ls='-', marker=None, label='persist')
plt.plot_date( ds_tc.time, np.mean(err2, axis=0, dtype=np.float64), ls='-',marker=None, label='persist anom')
plt.legend()
plt.ylabel('Mean of SST error ($^\circ$K)')
#
plt.subplot(313)
plt.plot_date( ds_tc.time, np.std(err1, axis=0, dtype=np.float64), ls='-', marker=None, label='persist')
plt.plot_date( ds_tc.time, np.std(err2, axis=0, dtype=np.float64), ls='-', marker=None, label='persist anom')
plt.legend()
plt.ylabel('STD Dev of SST error ($^\circ$K)')